# Constants & Libs

In [1]:
from pathlib import Path
import os
from tqdm import tqdm

In [2]:
os.getcwd()

'D:\\PythonProjects\\AML_test_task\\exploration\\notebooks'

In [3]:
PATH_TO_PROJECT = Path(os.getcwd()).parent.parent

In [4]:
os.chdir(PATH_TO_PROJECT)

In [5]:
PATH_TO_PROJECT

WindowsPath('D:/PythonProjects/AML_test_task')

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import re

from exploration.Modules.CheckerClass import Checker
from exploration.Modules.VisualisationClass import Vizualization

from feature_engine.encoding import MeanEncoder

from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from feature_engine.imputation import ArbitraryNumberImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from matplotlib import pyplot
from sklearn import cluster
from sklearn.preprocessing import MinMaxScaler

In [8]:
from boruta import BorutaPy

In [9]:
from sklearn.ensemble import IsolationForest

In [10]:
from feature_engine.imputation import AddMissingIndicator

In [11]:
from sklearn.model_selection import GridSearchCV
import joblib

In [12]:
# Imports from LightAutoML package
# from lightautoml.automl.presets.tabular_presets import TabularAutoML
# from lightautoml.tasks import Task

In [13]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,VotingClassifier

In [14]:
import lightgbm as lgb
import xgboost as xgb

In [15]:
PATH_TO_TRAIN_DATA = PATH_TO_PROJECT.joinpath('data/raw_data/train.csv')
PATH_TO_TEST_DATA = PATH_TO_PROJECT.joinpath('data/raw_data/test.csv')

In [16]:
PATH_TO_ARTEFACTS = PATH_TO_PROJECT.joinpath('exploration/artefacts')

In [17]:
PATH_TO_PREPARED_DATA = PATH_TO_PROJECT.joinpath('data/prepared_data')

# Read file

In [ ]:
train_data = pd.read_csv(PATH_TO_TRAIN_DATA)

In [ ]:
test_data = pd.read_csv(PATH_TO_TEST_DATA)

# EDA Train data

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
# совпадает ли Id с индексом
sum(test_data['Id'].values != test_data.index.values)

In [ ]:
# удаление столбца Id
test_data.drop(columns='Id', inplace=True)

## watch types, NaNs and etc.

In [ ]:
train_data.info()

In [ ]:
test_data.info()

### types

In [ ]:
for col in train_data.columns:
    print(col)
    print(train_data[col].map(type).unique(), '\n')

In [ ]:
for col in test_data.columns:
    print(col)
    print(test_data[col].map(type).unique(), '\n')

fact_dt только строковый тип, необходимо получить числа

столбцы в train и test в одинаковых форматах

### get days from fact_dt

In [ ]:
train_data

выделяем количество дней из строк в fact_dt

In [ ]:
def get_days(data: pd.DataFrame, col: str):
    day_numbers = []
    for amount_days in data[col]:
        days = int(re.search(r'[0-9]+', amount_days)[0])
        day_numbers.append(days)
    data[col] = day_numbers
    return data

In [ ]:
train_data = get_days(train_data, 'fact_dt')
test_data = get_days(test_data, 'fact_dt')

### describe features

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

макс. значения у entity_0015_ct, entity_0008_ct, entity_0001_ct  разлицаются в несколько раз, возможно выбросы

In [ ]:
train_data.groupby('target').agg(['mean', 'min', 'max'])

cтолбец entity_0016_rt полностью разделяет таргет

столбцы с rt видимо должны быть >= 0, но entity_0016_rt < 0, cкорее всего выброс

столбец entity_0007_ct есть только для таргета 1

In [ ]:
# сделаем идентификатор, чтобы посмотреть различия в значениях train и test
train_data['train_or_test'] = 'train'
test_data['train_or_test'] = 'test'

In [ ]:
pd.concat([train_data, test_data]).groupby('train_or_test').agg(['mean', 'min', 'max'])

cтолбец entity_0016_rt имеет абсолютно разные значение на train и test
возможно, стоит его вообще убрать или как-то преобразовать

часть фичей из эмбедингов могут полностью лежать в области < 0 (например emb_0020) на train, но при этом могут иметь значения > 0 на test, возможно выбросы

In [ ]:
# Vizualization(train_data).numTargetHist(target='target', 
#                                         save_folder=f'{PATH_TO_ARTEFACTS}/train_features_target.png')

In [ ]:
# Vizualization(pd.concat([train_data, test_data])).numTargetHist(
#     target='train_or_test', 
#     save_folder=f'{PATH_TO_ARTEFACTS}/train_test_features.png'
# )

небольшая доля user_id повторяется в train и test

небольшая доля fact_dt повторяется в train и test, но в целом в test значения больше (возможно там более новые значения)

In [ ]:
for col in train_data.columns:
    print(col)
    print(train_data[col].value_counts().iloc[:15], '\n')

таргет несбалансирован

In [ ]:
for col in test_data.columns:
    print(col)
    print(test_data[col].value_counts().iloc[:10], '\n')

категориальная переменная одна, категории одинаковые для train и test

#### change values

возможно допущена ошибка в данных entity_0016_rt, необходимо преобразовать, иначе невозможно использовать

In [ ]:
train_data['entity_0016_rt'] = np.where(train_data['entity_0016_rt'] < 0.5, 
                                        train_data['entity_0016_rt']+1,
                                        train_data['entity_0016_rt'])

In [ ]:
test_data['entity_0016_rt'] = np.where(test_data['entity_0016_rt'] < 0.5, 
                                       test_data['entity_0016_rt']+1,
                                       test_data['entity_0016_rt'])

преобразовываем категориальные переменные

In [ ]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [ ]:
train_data.shape, test_data.shape

#### drop features

In [ ]:
# убираем лишнюю колонку
train_data.drop(columns=['train_or_test_train'], inplace=True)
test_data.drop(columns=['train_or_test_test'], inplace=True)

### check duplicates

In [ ]:
# проверка на повторяющиеся строки
len(train_data[train_data.duplicated()]), len(test_data[test_data.duplicated()])

In [ ]:
# проверка на повторяющихся юзеров
train_data[train_data.duplicated(subset='user_id', keep=False)]

In [ ]:
# величина таргета для повторяющихся юзеров
train_data[train_data.duplicated(subset='user_id', keep=False)].mean()['target']

In [ ]:
test_data[test_data.duplicated(subset='user_id', keep=False)]

#### generate features

In [ ]:
train_data[train_data.duplicated(subset='user_id', keep=False)].mean()

повторяющихся юзеров можно использовать как допю фичу

In [ ]:
# определение списка повторяющихся юзеров
duplicated_users_train = train_data[train_data.duplicated(subset='user_id', keep=False)]['user_id'].unique()
duplicated_users_test = test_data[test_data.duplicated(subset='user_id', keep=False)]['user_id'].unique()

In [ ]:
# создание доп фичи repeat_user
train_data['repeat_user'] = np.where(train_data['user_id'].isin(duplicated_users_train), 
                                    1, 
                                    0)

test_data['repeat_user'] = np.where(test_data['user_id'].isin(duplicated_users_test), 
                                    1, 
                                    0)

In [ ]:
# определение уникальных пользователей в train и test
unique_id_train = set(train_data['user_id'].unique())
unique_id_test = set(test_data['user_id'].unique())                      

In [ ]:
# кол-во одинаковых пользователей в train и test
len(unique_id_train & unique_id_test)

In [ ]:
train_data[train_data['user_id'].isin(unique_id_test)]

#### drop features

In [ ]:
# удаление признака user_id
train_data.drop(columns='user_id', inplace=True)
test_data.drop(columns='user_id', inplace=True)

### check NaNs

In [ ]:
# определение процента пропусков в столбцах train
Checker(train_data).check_missing()

In [ ]:
# определение процента пропусков в столбцах test
Checker(test_data).check_missing()

видно, что пропуски в одних и тех же колонках, при чем у одних и тех же колонках процент пропусков одинаковый, в качестве доп. признаков можно использовать индикаторы пропусков

#### generate new features

In [ ]:
ami = AddMissingIndicator()

In [ ]:
train_data = ami.fit_transform(train_data)

In [ ]:
test_data = ami.fit_transform(test_data)

In [ ]:
train_data

проверим, на одних ли и тех же местах находятся пропуски, если да, то такие фичи одинаковы, часть из них надо удалить

In [ ]:
(train_data['entity_0015_ct_na'] == train_data['entity_0008_ct_na']).sum()

In [ ]:
(train_data['entity_0006_rt_na'] == train_data['entity_0012_ct_na']).sum()

#### drop features

In [ ]:
train_data.drop(columns=['entity_0008_ct_na', 'entity_0012_ct_na', 'entity_0014_rt_na'], inplace=True)
test_data.drop(columns=['entity_0008_ct_na', 'entity_0012_ct_na', 'entity_0014_rt_na'], inplace=True)

### correlations

оценим степень корреляции признаков между собой

In [ ]:
corr_data = Checker(train_data).featureCorrelation(target='target', corr_edge=0.7)

In [ ]:
corr_data['negativeCorr']

In [ ]:
corr_data['positiveCorr']

### find outliers

добавим в качестве признаков индикаторы выбросов

In [ ]:
train_data.columns

In [ ]:
cols_to_drop=['target', 'entity_0017_cd_a', 'entity_0017_cd_b', 'entity_0017_cd_c',
       'entity_0017_cd_d', 'repeat_user', 'entity_0006_rt_na',
       'entity_0007_ct_na', 'entity_0009_rt_na', 'entity_0010_rt_na',
       'entity_0015_ct_na']

In [ ]:
features_train_for_outliers = train_data.drop(columns=cols_to_drop).copy()
features_test_for_outliers = test_data.drop(columns=cols_to_drop[1:]).copy()

In [ ]:
features_train_for_outliers.fillna(-9999, inplace=True)
features_test_for_outliers.fillna(-9999, inplace=True)

In [ ]:
features_train_for_outliers

In [ ]:
features_test_for_outliers

In [ ]:
outliers_finder=IsolationForest(random_state=42, verbose=0)

In [ ]:
for col in tqdm(features_train_for_outliers.columns):
    outliers_finder.fit(features_train_for_outliers[[col]])
    anomalies_detected_train = outliers_finder.predict(features_train_for_outliers[[col]])
    anomalies_detected_test = outliers_finder.predict(features_test_for_outliers[[col]])
    train_data[f'{col}_outlier'] = anomalies_detected_train
    test_data[f'{col}_outlier'] = anomalies_detected_test

In [ ]:
train_data

In [ ]:
test_data

### generate math features

генерирование параметров на основе мат функций

In [ ]:
embeding_columns = [col for col in train_data.columns if 'emb' in col and 'outlier' not in col]
embeding_columns

In [ ]:
count_columns = [col for col in train_data.columns if 'ct' in col]
count_columns

In [ ]:
ratio_columns = [col for col in train_data.columns if 'rt' in col]
ratio_columns

In [ ]:
def math_embeding_columns(data, columns):
    embedings_features = data[columns].copy()
    embedings_features['mul_emb'] = embedings_features.prod(axis=1)
    embedings_features['sum_emb'] = embedings_features.sum(axis=1)
    embedings_features['sum_abs_emb'] = abs(embedings_features).sum(axis=1)
    embedings_features.drop(columns=columns, inplace=True)
    return embedings_features

In [ ]:
def math_count_columns(data, columns):
    count_features = data[columns].copy()
    count_features['mul_ct'] = count_features.fillna(1).prod(axis=1)
    count_features['sum_ct'] = count_features.fillna(0).sum(axis=1)
    count_features.drop(columns=columns, inplace=True)
    return count_features

In [ ]:
def math_ratio_columns(data, columns):
    ratio_features = data[columns].copy()
    ratio_features['mul_rt'] = ratio_features.fillna(1).prod(axis=1)
    ratio_features['sum_rt'] = ratio_features.fillna(0).sum(axis=1)
    ratio_features.drop(columns=columns, inplace=True)
    return ratio_features

In [ ]:
train_data[ratio_columns]

In [ ]:
train_data_ratio = math_ratio_columns(train_data, ratio_columns)
test_data_ratio = math_ratio_columns(test_data, ratio_columns)

In [ ]:
train_data_math = math_count_columns(train_data, count_columns)
test_data_math = math_count_columns(test_data, count_columns)

In [ ]:
train_data_embeding = math_embeding_columns(train_data, embeding_columns)
test_data_embeding = math_embeding_columns(test_data, embeding_columns)

In [ ]:
train_data = pd.concat([train_data, train_data_ratio, train_data_math, train_data_embeding], axis=1)
test_data = pd.concat([test_data, test_data_ratio, test_data_math, test_data_embeding], axis=1)

In [ ]:
test_data

### feature selection

через алгоритм boruta отберем важные признаки

In [ ]:
features_train_for_feature_selection = train_data.drop(columns='target').copy()
target_train_for_feature_selection = train_data['target']

In [ ]:
features_train_for_feature_selection

In [ ]:
features_train_for_feature_selection.fillna(-9999, inplace=True)

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=10)

In [ ]:
feature_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [ ]:
feature_selector.fit(features_train_for_feature_selection.values, 
                     target_train_for_feature_selection.values)

In [ ]:
feature_selector.support_

In [ ]:
len(feature_selector.support_)

In [ ]:
need_cols = []
for col, status in zip(features_train_for_feature_selection.columns, feature_selector.support_):
    if status==True:
        need_cols.append(col)

In [ ]:
# check ranking of features
feature_selector.ranking_

In [ ]:
need_cols

### make features_train, features_test, target_train and save

In [ ]:
features_train = train_data[need_cols]
target_train = train_data['target']

In [ ]:
features_test = test_data[need_cols]

In [ ]:
features_train.to_pickle(f'{PATH_TO_PREPARED_DATA}/features_train.pickle')
features_test.to_pickle(f'{PATH_TO_PREPARED_DATA}/features_test.pickle')
target_train.to_pickle(f'{PATH_TO_PREPARED_DATA}/target_train.pickle')

# Models

## load existing model

In [ ]:
best_model = joblib.load(f'{PATH_TO_ARTEFACTS}/model.pickle')

In [ ]:
best_model.get_params()

## read prepared files

In [ ]:
features_train = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/features_train.pickle')
features_test = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/features_test.pickle')
target_train = pd.read_pickle(f'{PATH_TO_PREPARED_DATA}/target_train.pickle')

In [ ]:
features_train

In [ ]:
features_test

## baseline model

проба стандартных моделей

In [ ]:
pipeline_stages = []

pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))
pipeline_stages.append(("model", CatBoostClassifier(verbose=0)))

pipeline = Pipeline(pipeline_stages)

In [ ]:
param_grid = [
    {
        "imputer": [None],
        "model": [CatBoostClassifier(n_estimators=1000, random_state=42, verbose=0, max_depth=10)],
    },
    {
        "imputer": [ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [lgb.LGBMClassifier(n_estimators=1000, max_depth=10)],
    },
    {
        "imputer": [ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [xgb.XGBClassifier(n_estimators=1000, max_depth=10)],
    },
]

In [ ]:
tscv = StratifiedKFold(n_splits=5)

In [ ]:
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring="roc_auc", cv=tscv, verbose=2)

In [ ]:
grid.fit(features_train, target_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.cv_results_

### voting models

эта модель, давшая один из 2 лучших сабмитов

In [ ]:
estimator = [
    ('m1', CatBoostClassifier(n_estimators=1000, max_depth=10)),
    ('m4', (lgb.LGBMClassifier(n_estimators=1000, max_depth=10))),
    ('m5', xgb.XGBClassifier(n_estimators=1000, max_depth=10)),
]

In [ ]:
pipeline_stages = []

pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))
pipeline_stages.append(('model', VotingClassifier(estimators = estimator, voting='soft')))

# Собираем все шаги в пайплайн
pipeline = Pipeline(pipeline_stages)

In [ ]:
tscv = StratifiedKFold(n_splits=5)

In [ ]:
metric_results = []
for train_idx, valid_idx in tscv.split(features_train, target_train):
    # разбиваем по фолдам
    X_train, X_test = features_train.iloc[train_idx], features_train.iloc[valid_idx]
    y_train, y_test = target_train.iloc[train_idx], target_train.iloc[valid_idx]
    # обучаем модель и считаем метрики
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)
    metric_results.append(roc_auc_score(y_test, y_pred[:, 1]))
    print(roc_auc_score(y_test, y_pred[:, 1]))
print(np.mean(metric_results))

## find best model

выбор модели с подбором гиперпараметров (дает один из лучших 2 сабмитов)

In [ ]:
pipeline_stages = []

pipeline_stages.append(("imputer", ArbitraryNumberImputer(arbitrary_number=-9999)))
pipeline_stages.append(("scaler", StandardScaler()))
pipeline_stages.append(("model", CatBoostClassifier(verbose=0)))

pipeline = Pipeline(pipeline_stages)

In [ ]:
param_grid = [
    {
        "imputer": [SimpleImputer(strategy='median'), ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [CatBoostClassifier(random_state=42, verbose=0)],
        "model__n_estimators": [500, 1000],
        "model__max_depth": [6, 8, 10, 12],
        'model__loss_function': ['Logloss', 'CrossEntropy'],
        "model__l2_leaf_reg": [3, 4, 5], 
        "model__min_data_in_leaf": [1, 3, 5],
        "scaler": [None]
    },
    {
        "imputer": [SimpleImputer(strategy='median'), ArbitraryNumberImputer(arbitrary_number=-9999)],
        "model": [LogisticRegression(random_state=42)],
        "model__C": [0.001, 0.01, 0.1, 1, 10, 100],
        "model__penalty":['none', 'elasticnet', 'l1', 'l2'],
        "model__solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        "scaler": [StandardScaler()]
    },
]

In [ ]:
tscv = StratifiedKFold(n_splits=5)

In [ ]:
grid = GridSearchCV(pipeline, param_grid=param_grid, scoring="roc_auc", cv=tscv, verbose=2)

In [ ]:
grid.fit(features_train, target_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.cv_results_

# Make submission

In [ ]:
# pipeline = grid.best_estimator_

In [ ]:
pipeline.fit(features_train, target_train)

In [ ]:
y_pred = pipeline.predict_proba(features_test)

In [ ]:
y_pred = y_pred[:, 1]

In [ ]:
len(y_pred), sum(y_pred)

In [ ]:
calibrated_clf = CalibratedClassifierCV(pipeline, cv=StratifiedKFold(n_splits=5))
calibrated_clf.fit(features_train, target_train)
y_pred_proba = calibrated_clf.predict_proba(features_test)
y_pred_proba = y_pred_proba[:, 1]

In [ ]:
submit_data = pd.DataFrame(data=y_pred, columns=['Predicted'])

In [ ]:
submit_data.to_csv(f'{PATH_TO_PREPARED_DATA}/submit.csv', index_label='Id', sep=",")